In [300]:
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [301]:
data = pd.read_csv("./DATASET/data.csv")
data.head()


,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,...,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v,pose
0,0.441700,0.694201,-0.096439,0.997566,0.414700,0.543821,-0.350309,0.997411,0.421533,0.537008,...,0.413311,0.825927,0.860303,-0.317801,0.972257,0.809430,0.840739,0.287854,0.439358,0
1,0.375681,0.698162,-0.107649,0.990680,0.413994,0.580372,-0.404412,0.994068,0.403408,0.545868,...,0.457589,0.770906,0.910949,-0.242096,0.977082,0.758387,0.892177,0.341211,0.453720,0
2,0.415092,0.678437,-0.035257,0.999260,0.378349,0.525345,-0.256553,0.998898,0.385751,0.523129,...,0.467491,0.833564,0.854598,-0.030145,0.969620,0.821841,0.834830,0.436641,0.540311,0
3,0.572214,0.532797,-0.121105,0.999968,0.547253,0.469890,-0.273464,0.999882,0.518167,0.443134,...,0.652442,0.765417,0.854516,-0.209655,0.964131,0.530205,0.734180,0.113064,0.646947,0
4,0.607926,0.708455,-0.065062,0.999852,0.622632,0.545505,0.125140,0.998698,0.621549,0.556630,...,0.997850,0.382435,0.913913,0.310582,0.849778,0.372350,0.932190,-0.056707,0.996489,0


In [302]:
features = data.drop("pose", axis=1)
target = data["pose"]

Xtrain, xtest, Ytrain, ytest = train_test_split(features, target, test_size=0.2, random_state=1234)


In [303]:
model = RandomForestClassifier()
model.fit(Xtrain, Ytrain)


RandomForestClassifier()

In [304]:
from sklearn.metrics import classification_report

print(classification_report(ytest, model.predict(xtest)))


              precision    recall  f1-score   support

           0       0.95      0.90      0.92        40
           1       0.95      0.92      0.93        38
           2       0.89      0.96      0.92        51
           3       0.94      1.00      0.97        29
           4       1.00      0.94      0.97        51

    accuracy                           0.94       209
   macro avg       0.94      0.94      0.94       209
weighted avg       0.94      0.94      0.94       209



In [305]:
import mediapipe as mp
import mediapipe as mp
import pandas as pd
import numpy as np
import cv2
import os


In [306]:
mp_pose = mp.solutions.pose


In [307]:
landmarks_points = {
    "nose": 0,
    "left_shoulder": 11, "right_shoulder": 12,
    "left_elbow": 13, "right_elbow": 14,
    "left_wrist": 15, "right_wrist": 16,
    "left_hip": 23, "right_hip": 24,
    "left_knee": 25, "right_knee": 26,
    "left_ankle": 27, "right_ankle": 28,
    "left_heel": 29, "right_heel": 30,
    "left_foot_index": 31, "right_foot_index": 32,
}

col_names = []
for i in range(len(landmarks_points.keys())):
    name = list(landmarks_points.keys())[i]
    col_names.append(name + "_x")
    col_names.append(name + "_y")
    col_names.append(name + "_z")
    col_names.append(name + "_v")


In [308]:
full_list = []

main_dir = "./DATASET/TEST"
pose_dir_list = os.listdir(main_dir)

for j in range(4, len(pose_dir_list)):
    dir_list = os.listdir(f"{main_dir}/{pose_dir_list[j]}")

    for i in range(108, len(dir_list)):

        pre_list = []

        with mp_pose.Pose(static_image_mode=True, enable_segmentation=True) as pose:
            image = cv2.imread(f"{main_dir}/{pose_dir_list[j]}/{dir_list[i]}")
            result = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            try:
                predict = True
                for landmarks in result.pose_landmarks.landmark:
                    pre_list.append(landmarks)
            except AttributeError:
                print(
                    f"No points on {main_dir}/{pose_dir_list[j]}/{dir_list[i]}")
                predict = False

        if predict == True:
            gen1116 = np.array([
                [
                    pre_list[k].x,
                    pre_list[k].y,
                    pre_list[k].z,
                    pre_list[k].visibility
                ] for k in range(11, 17)
            ]).flatten().tolist()

            gen2333 = np.array([
                [
                    pre_list[k].x,
                    pre_list[k].y,
                    pre_list[k].z,
                    pre_list[k].visibility
                ] for k in range(23, 33)
            ]).flatten().tolist()

            gen1116.extend(gen2333)

            all_list = [
                pre_list[0].x,
                pre_list[0].y,
                pre_list[0].z,
                pre_list[0].visibility,
            ]

            all_list.extend(gen1116)


In [309]:
data_x = pd.DataFrame([all_list], columns=col_names)


In [310]:
pred = model.predict(data_x)


In [311]:
pose_to_class_name = {
    0: "downdog",
    1: "goddess",
    2: "plank",
    3: "tree",
    4: "warrior2"
}


In [312]:
print(f"The yoga pose is: {pose_to_class_name[pred[0]]}")


The yoga pose is: warrior2
